In [1]:
import requests
import json

TRAPI Documentation: https://github.com/NCATSTranslator/ReasonerAPI

Most TRAPI documents contain a `message` key.  Within that `message` are a `query_graph` denoting the user query,
a `knowledge_graph` consisting of the union of all nodes and edges that match the `query_graph` pattern, and a list of `results` that bind `query_graph` elements to `knowledge_graph` elements.

The following message contains only a `query_graph`.  This query graph consists of 3 nodes connected together in a line.   Two of the nodes (`n00` and `n02`) have specified identifiers, while the middle node of the line does not.  Rather the middle node has a list of `categories` that are acceptable.

This query asks "Find me a Biological Process or Activity, or a Gene, or a Pathway that is related to both `PUBCHEM.COMPOUND:644073` (Buprenorphine) and `HP:0001337` (Tremor).

In [16]:
query={
    "message": {
      "query_graph": {
        "edges": {
          "e00": {
            "subject": "n00",
              "object": "n01",
          "predicates":["biolink:related_to"]
          },
          "e01": {
            "subject": "n01",
              "object": "n02",
          "predicates":["biolink:related_to"]
          }
        },
        "nodes": {
          "n00": {
            "ids": ["PUBCHEM.COMPOUND:644073"],
            "categories": ["biolink:ChemicalEntity"]
          },
          "n01": {
              "categories": ["biolink:BiologicalProcessOrActivity","biolink:Gene","biolink:Pathway"]
          },
          "n02": {
            "ids": ["HP:0001337"],
            "categories": ["biolink:DiseaseOrPhenotypicFeature"]
          }
        }
      }
    }
  }


This query can be sent to various components of Translator as needed.  It can be sent directly to the ROBOKOP database like this:

In [17]:
robokop_submit_url = "https://automat.renci.org/robokopkg/1.3/query"
response = requests.post(robokop_submit_url,json=query)

In [18]:
print(response.status_code)

200


In [19]:
print(len(response.json()['message']['results']))

7


You will have noticed that the specified nodes `n00` and `n02` are defined in terms of identifiers, not names.  All nodes and edges in Translator queries and results are defined by CURIES (Compact URIs).  There are two tools in Translator to help navigate identifiers and queries.   

The name-resolver: https://name-resolution-sri.renci.org/docs has a lookup function that can take a string and return potential identifiers.  Here, we look up the string "alzheimer"

In [16]:
results = requests.post('https://name-resolution-sri.renci.org/lookup?string=alzheimer&offset=0&limit=10')

In [18]:
print(json.dumps(results.json(),indent=4))

{
    "MONDO:0004975": [
        "Alzheimer",
        "Alzheimers",
        "Alzheimer's",
        "ALZHEIMER DIS",
        "ALZHEIMERS DIS",
        "Alzheimer Disease",
        "ALZHEIMER DISEASE",
        "Alzheimer disease",
        "Alzheimer Dementia",
        "Alzheimers disease",
        "DEMENTIA ALZHEIMER",
        "Alzheimer syndrome",
        "alzheimers disease",
        "Disease, Alzheimer",
        "Alzheimer Syndrome",
        "alzheimer diseases",
        "Disease;Alzheimers",
        "Alzheimers Disease",
        "Alzheimer dementia",
        "Alzheimer's Disease",
        "Alzheimer's disease",
        "Alzheimers Dementia",
        "Alzheimers dementia",
        "ALZHEIMER'S DISEASE",
        "alzheimer's disease",
        "Alzheimer Sclerosis",
        "Alzheimer Dementias",
        "Alzheimer sclerosis",
        "Dementia, Alzheimer",
        "alzheimers dementia",
        "dementia alzheimers",
        "sclerosis; Alzheimer",
        "Disease, Alzheimer's",
     

The node normalizer (https://nodenormalization-sri.renci.org/docs) takes CURIES and returns all other CURIES that are synonymous with the input. It also returns labels for the node, the biolink classes of the node, and often the information content of the node.

In [21]:
nn_query = {
  "curies": [
    "MONDO:0004975",
  ],
  "conflate": True
}
results = requests.post('https://nodenormalization-sri.renci.org/get_normalized_nodes',json=nn_query)

In [22]:
print(json.dumps(results.json(),indent=4))

{
    "MONDO:0004975": {
        "id": {
            "identifier": "MONDO:0004975",
            "label": "Alzheimer disease"
        },
        "equivalent_identifiers": [
            {
                "identifier": "MONDO:0004975",
                "label": "Alzheimer disease"
            },
            {
                "identifier": "DOID:10652",
                "label": "Alzheimer's disease"
            },
            {
                "identifier": "EFO:0000249",
                "label": "Alzheimer's disease"
            },
            {
                "identifier": "UMLS:C0002395",
                "label": "Alzheimer's Disease"
            },
            {
                "identifier": "UMLS:C0011265",
                "label": "Presenile dementia"
            },
            {
                "identifier": "UMLS:C0276496",
                "label": "Familial Alzheimer Disease (FAD)"
            },
            {
                "identifier": "UMLS:C0494463",
                "label":

The results above are just database matches, there are no scores or other additions.  You can instead send the TRAPI to the robokop application (rather than just to the graph)

In [27]:
ara_robokop_submit_url = "https://aragorn.renci.org/robokop/query"
response = requests.post(ara_robokop_submit_url,json=query)

In [28]:
response.status_code

200

In [29]:
len(response.json()['message']['results'])

14

In [30]:
for result in response.json()['message']['results']:
    print(result['score'])

0.17813345082933515
0.1477493684070679
0.13196624865367593
0.11703906793268536
0.11050745673684113
0.11020805622029194
0.10693143597045583
0.10636103810501632
0.1060730134897604
0.10201048277292121
0.10139427809144123
0.1010853670864079
0.10108536708640783
0.10108536708640783


You can also bypass TRAPI entirely and just use cypher to talk to the graph.  There are two instances.  There is one at http://robokopkg.renci.org which has a cypher browser on it, or you can write cypher and post it there.

In [31]:
from neo4j import GraphDatabase

In [32]:
pw=''
driver = GraphDatabase.driver('bolt://robokopkg.renci.org:7687', auth=('neo4j', pw))

In [36]:
cypher = f'MATCH (a:`biolink:Gene`) RETURN a LIMIT 1'

with driver.session() as session:
    results = session.run(cypher)
    for result in results:
        print(result)



<Record a=<Node id=5044448 labels=frozenset({'biolink:OntologyClass', 'biolink:ThingWithTaxon', 'biolink:ChemicalEntityOrGeneOrGeneProduct', 'biolink:GeneOrGeneProduct', 'biolink:GenomicEntity', 'biolink:Gene', 'biolink:NamedThing', 'biolink:Entity', 'biolink:BiologicalEntity', 'biolink:MacromolecularMachineMixin', 'biolink:PhysicalEssence', 'biolink:PhysicalEssenceOrOccurrent'}) properties={'name': 'ENSACAG00000017044', 'id': 'ENSEMBL:ENSACAG00000017044', 'equivalent_identifiers': ['ENSEMBL:ENSACAG00000017044']}>>


You can also send the cypher through the automat interface instead:

In [39]:
j = {'query': cypher}
results = requests.post('https://automat.renci.org/robokopkg/cypher',json=j)

In [41]:
print(results.json())

{'results': [{'columns': ['a'], 'data': [{'row': [{'name': 'ENSACAG00000017044', 'id': 'ENSEMBL:ENSACAG00000017044', 'equivalent_identifiers': ['ENSEMBL:ENSACAG00000017044']}], 'meta': [{'id': 5044448, 'type': 'node', 'deleted': False}]}]}], 'errors': []}
